# Import Libraries

In [ ]:
import io
import pandas as pd
import numpy as np
import datetime as dt
import statsmodels.api as sm

from google.colab import files
from google.colab import drive
drive.mount('drive')

In [ ]:
uploaded = files.upload()

# Import and Aggregate Data

## Analyst Coverage

In [ ]:
cv = pd.read_csv(io.BytesIO(uploaded['af.csv']))
cv['Date'] = pd.to_datetime(cv['ANNDATS'])
cv['ANALYS']=pd.Categorical(cv['ANALYST'])
cv['Date']= pd.to_datetime(cv['Date'])
cv['year'] = cv['Date'].dt.to_period('Y')
ac = cv.groupby(by=['OFTIC',"year"],
                as_index=False).count()
ac = ac[['OFTIC','year','ANALYST']]
ac

## Governance

In [ ]:
gov = pd.read_csv(io.BytesIO(uploaded['governance.csv']))
gov = gov[['Date','Company','Ticker','governance_score']]
gov['Date'] = pd.to_datetime(gov['Date'])
gov['year'] = pd.to_datetime(gov['Date']).dt.to_period('Y')
gov

## Control Variables

In [ ]:
controls = pd.read_csv(io.BytesIO(uploaded['ratios.csv']))
controls['date'] = pd.to_datetime(controls['date'])
controls['year'] = pd.to_datetime(controls['date']).dt.to_period('Y')
controls = controls.fillna(1)

controls

## Real Activities Earnings Management

In [ ]:
raem = pd.read_csv(io.BytesIO(uploaded['raem.csv']))
raem['datadate'] = pd.to_datetime(raem['datadate'] )
raem['year'] = raem['datadate'].dt.to_period('Y')
raem = raem.fillna(0)
raem

# Generating Accrual Management

In [ ]:
accrued = pd.read_csv(io.BytesIO(uploaded['accrued_data.csv']))
accrued = accrued.fillna(1)
accrued

In [ ]:
accrued['tic']  = pd.Categorical(accrued['tic'],ordered=True)
companies = list(accrued['tic'].dtypes.categories)
companies

In [ ]:
ams = []
for comp in companies:
  accruals=[]
  X1=[]
  X2=[]
  X3=[]
  X4=[]
  for index, row in accrued.iterrows():
    if row['tic'] == comp and row['fyear']!=2011:
      index_1 = index-1
      pv_asset = accrued.at[index_1,'at']
      pv_rev = accrued.at[index_1,'revt']
      pv_rec = accrued.at[index_1,'rect']
      rev_change = row['revt']-pv_rev
      rec_change = row['rect']-pv_rec

      accrual = (row['ni']-row['oancf'])/pv_asset

      x1 = 1/pv_asset
      x2 = (rev_change-rec_change)/(row['at']-1)
      x3 = row['ppegt']/pv_asset
      x4 = row['intan']/pv_asset
      
    elif row['tic'] == comp and row['fyear']==2011: 
      x1=0
      x2=0
      x3=0
      x4=0
      accrual=0

    else:
      continue
    X1.append(x1)
    X2.append(x2)
    X3.append(x3)
    X4.append(x4)
    accruals.append(accrual)

  X = np.array([X1,X2,X3,X4]).T
  X = sm.add_constant(X)
  model = sm.OLS(accruals,X).fit()

  am = model.resid

  for a in am:
    ams.append(a)

In [ ]:
ams_all = pd.DataFrame({"tic":accrued['tic'],
                        "date":accrued['datadate'],
                        'am':ams})
ams_all['year']=pd.to_datetime(ams_all['date']).dt.to_period('Y')
ams_all

# Generating Abnormal OCF, COGS, INVT and PROD

In [ ]:
# OANCF -- Operating Activities Net Cash Flow (OANCF)
raem['disexp'] = raem['xad']+raem['xrd']+raem['xsga']
raem['year']=pd.to_datetime(raem['datadate']).dt.to_period('Y')
raem['year'] = raem['year'].astype(str)
raem

In [ ]:
len(aocfs)

### Abnormal OCF

In [ ]:
aocfs=[]

for comp in companies:
  ocf = []
  X1 = []
  X2 = []
  X3 = []

  for index, row in raem.iterrows():
    if row['tic'] == comp and row['year'] != '2011':
      index_1 = index-1
      sales_t_1 = raem.at[index_1,'revt']
      asset_t_1 = raem.at[index_1,'at']
      sales_change = row['revt']-sales_t_1
      #independent variables
      x1 = 1/asset_t_1
      x2 = row['revt']/asset_t_1
      x3 = sales_t_1/asset_t_1
      #dependent variable
      y = row['oancf']/asset_t_1
    elif row['tic'] == comp and row['year']=='2011': 
      y=0
      x1=0
      x2=0
      x3=0
    else:
      continue

    X1.append(x1)
    X2.append(x2)
    X3.append(x3)
    ocf.append(y)

  X = np.array([X1,X2,X3]).T
  X = sm.add_constant(X)
  model_ocf = sm.OLS(ocf,X).fit()
  aocf = model_ocf.resid

  for a in aocf:
    aocfs.append(a)

### Abnormal COGS

In [ ]:
acogss=[]
beta0s_acogs = []
beta1s_acogs = []
beta2s_acogs = []

for comp in companies:
  cogs = []
  X1 = []
  X2 = []

  for index, row in raem.iterrows():
    if row['tic'] == comp and row['year'] != '2011':
      index_1 = index-1
      asset_t_1 = raem.at[index_1,'at']
      #independent variables
      x1 = 1/asset_t_1
      x2 = row['revt']/asset_t_1
      #dependent variable
      y = row['cogs']/asset_t_1
    elif row['tic'] == comp and row['year']=='2011': 
      y=0
      x1=0
      x2=0
      x3=0
    else:
      continue

    X1.append(x1)
    X2.append(x2)

    cogs.append(y)

  X = np.array([X1,X2]).T
  X = sm.add_constant(X)
  model_cogs = sm.OLS(cogs,X).fit()
  acogs = model_cogs.resid
  beta0s_acogs.append(model_cogs.params[0].astype(float))
  beta1s_acogs.append(model_cogs.params[1].astype(float))
  beta2s_acogs.append(model_cogs.params[2].astype(float))

  for a in acogs:
    acogss.append(a)

### Abnormal Discretionary Expenses

In [ ]:
adiss=[]
beta0s_dis = []
beta1s_dis = []
beta2s_dis = []

for comp in companies:
  dis = []
  X1 = []
  X2 = []

  for index, row in raem.iterrows():
    if row['tic'] == comp and row['year'] != '2011':
      index_1 = index-1
      asset_t_1 = raem.at[index_1,'at']
      #independent variables
      x1 = 1/asset_t_1
      x2 = row['revt']/asset_t_1
      #dependent variable
      y = row['disexp']/asset_t_1
    elif row['tic'] == comp and row['year']=='2011': 
      y=0
      x1=0
      x2=0
      x3=0
    else:
      continue

    X1.append(x1)
    X2.append(x2)

    dis.append(y)

  X = np.array([X1,X2]).T
  X = sm.add_constant(X)
  model_dis = sm.OLS(dis,X).fit()
  adis = model_dis.resid
  beta0s_dis.append(model_dis.params[0].astype(float))
  beta1s_dis.append(model_dis.params[1].astype(float))
  beta2s_dis.append(model_dis.params[2].astype(float))

  for a in adis:
    adiss.append(a)

### Abnormal Inventory Changes

In [ ]:
ainvts=[]
beta0s_invt = []
beta1s_invt = []
beta2s_invt = []
beta3s_invt = []

for comp in companies:
  invt = []
  X1 = []
  X2 = []
  X3 = []
  for index, row in raem.iterrows():
    if row['tic'] == comp and row['year'] != '2011':
      index_1 = index-1
      asset_t_1 = raem.at[index_1,'at']
      sales_t_1 = raem.at[index_1,'revt']
      invt_t_1 = raem.at[index_1,'invt']
      #independent variables
      x1 = 1/asset_t_1
      x2 = row['revt']/asset_t_1
      x3 = (row['revt']-sales_t_1)/asset_t_1
      #dependent variable
      y = (row['invt']-invt_t_1)/asset_t_1

    elif row['tic'] == comp and row['year']=='2011': 
      y=0
      x1=0
      x2=0
      x3=0
    else:
      continue

    X1.append(x1)
    X2.append(x2)
    X3.append(x3)
    invt.append(y)

  X = np.array([X1,X2,X3]).T
  X = sm.add_constant(X)

  model_invt = sm.OLS(invt,X).fit()
  ainvt = model_invt.resid

  beta0s_invt.append(model_invt.params[0].astype(float))
  beta1s_invt.append(model_invt.params[1].astype(float))
  beta2s_invt.append(model_invt.params[2].astype(float))
  beta3s_invt.append(model_invt.params[3].astype(float))

  for a in ainvt:
    ainvts.append(a)

### Abnormal Production costs

In [ ]:
aprods=[]
beta0s_prod = []
beta1s_prod = []
beta2s_prod = []
beta3s_prod = []

for comp in companies:
  prod = []
  X1 = []
  X2 = []
  X3 = []
  for index, row in raem.iterrows():
    if row['tic'] == comp and row['year'] != '2011':
      index_1 = index-1
      asset_t_1 = raem.at[index_1,'at']
      sales_t_1 = raem.at[index_1,'revt']
      invt_t_1 = raem.at[index_1,'invt']
      invt_delta = row['invt']-invt_t_1
      prodc = invt_delta + row['cogs']

      #independent variables
      x1 = 1/asset_t_1
      x2 = row['revt']/asset_t_1
      x3 = (row['revt']-sales_t_1)/asset_t_1

      #dependent variable
      y = prodc/asset_t_1

    elif row['tic'] == comp and row['year']=='2011': 
      y=0
      x1=0
      x2=0
      x3=0
    else:
      continue

    X1.append(x1)
    X2.append(x2)
    X3.append(x3)
    prod.append(y)

  X = np.array([X1,X2,X3]).T
  X = sm.add_constant(X)

  model_prod = sm.OLS(prod,X).fit()
  aprod = model_prod.resid

  beta0s_prod.append(model_prod.params[0].astype(float))
  beta1s_prod.append(model_prod.params[1].astype(float))
  beta2s_prod.append(model_prod.params[2].astype(float))
  beta3s_prod.append(model_prod.params[3].astype(float))

  for a in aprod:
    aprods.append(a)

### Aggregate all abnormals

In [ ]:
abnormals = pd.DataFrame({'tic':raem['tic'],
                          'date':raem['datadate'],
                          'abnormal_ocf':aocfs,
                          'abnormal_cogs':acogss,
                          'abnormal_disexp':adiss,
                          'abnormal_inv':ainvts,
                          'abnormal_prod':aprods})
abnormals['year'] = pd. to_datetime(abnormals['date']).dt.to_period('Y')
abnormals

# Aggregate independent variables

In [ ]:
size = raem[['tic','datadate','year','at']]
size['size'] = np.log(size['at'])
size

In [ ]:
controls['year'] = pd.to_datetime(controls['date']).dt.to_period('Y')
size['year'] = pd.to_datetime(size['datadate']).dt.to_period('Y')

iv = pd.merge(left = controls,
              right = size,
              right_on = ['tic','year'],
              left_on=['TICKER','year'],
              how = 'right')
iv = iv.drop(['date','TICKER'],axis = 1)
iv = iv.dropna()
iv

In [ ]:
iv_new = pd.merge(left = iv,
                  right = ac,
                  left_on = ['tic','year'],
                  right_on = ['OFTIC','year'])
iv_new

In [ ]:
iv_new = iv_new.fillna(0)

In [ ]:
iv_new['tic'] = iv_new['tic'].astype(str)
abnormals['tic'] = abnormals['tic'].astype(str)

In [ ]:
final = pd.merge(left = abnormals,
                 right = iv_new,
                 left_on=['tic','year'],
                 right_on=['tic','year'])
final

In [ ]:
final_1 = pd.merge(left = final,
                   right = gov,
                   left_on = ['tic','year'],
                   right_on = ['Ticker','year'])
final_1 = final_1.drop(['datadate','date','Ticker','Date','at','Company'],axis=1)
final_1['coverage'] = np.log(final_1['ANALYST']+1)

In [ ]:
final_data = pd.get_dummies(final_1,columns=['year'])
final_data['abnormal_ocf']=final_data['abnormal_ocf']*100
final_data['abnormal_cogs']=final_data['abnormal_cogs']*100
final_data['abnormal_disexp']=final_data['abnormal_disexp']*100
final_data['abnormal_inv']=final_data['abnormal_inv']*100
final_data['abnormal_prod']=final_data['abnormal_prod']*100
final_data

In [ ]:
X = final_data[['coverage','ptb','roa','size','cycle',
                'year_2011','year_2012','year_2013',
                'year_2014','year_2015','year_2016',
                'year_2017']]
X = sm.add_constant(X)
y = abs(final_data['abnormal_ocf'])
model_1 = sm.OLS(y,X).fit()
#print(model_1.summary())

In [ ]:
reg_ocf = pd.concat((model_1.params, model_1.tvalues,model_1.pvalues), axis=1)

reg_ocf = reg_ocf.rename(columns={0: 'Coef.', 
                                  1: '|t|',
                                  2:'p-value'})

In [ ]:
X = final_data[['coverage','ptb','roa','size','cycle',
                'year_2011','year_2012','year_2013',
                'year_2014','year_2015','year_2016',
                'year_2017']]
X = sm.add_constant(X)
y = abs(final_data['abnormal_disexp'])
model_2 = sm.OLS(y,X).fit()
#print(model_2.summary())

In [ ]:
reg_disexp = pd.concat((model_2.params, model_2.tvalues,model_2.pvalues), axis=1)

reg_disexp = reg_disexp.rename(columns={0: 'Coef.', 
                                  1: 't',
                                  2:'p>|t|'})

In [ ]:
X = final_data[['coverage','ptb','roa','size','cycle',
                'year_2011','year_2012','year_2013',
                'year_2014','year_2015','year_2016',
                'year_2017']]
X = sm.add_constant(X)
y = abs(final_data['abnormal_prod'])
model_3 = sm.OLS(y,X).fit()
#print(model_3.summary())

In [ ]:
reg_prod = pd.concat((model_3.params, model_3.tvalues,model_3.pvalues), axis=1)

reg_prod = reg_prod.rename(columns={0: 'Coef.', 
                                  1: 't',
                                  2:'p>|t|'})

In [ ]:
X = final_data[['coverage','ptb','roa','size','cycle',
                'year_2011','year_2012','year_2013',
                'year_2014','year_2015','year_2016',
                'year_2017']]
X = sm.add_constant(X)
y = abs(final_data['abnormal_inv'])
model_4 = sm.OLS(y,X).fit()
#print(model_4.summary())

In [ ]:
reg_inv = pd.concat((model_4.params, model_4.tvalues,model_4.pvalues), axis=1)

reg_inv = reg_inv.rename(columns={0: 'Coef.', 
                                  1: 't',
                                  2:'p>|t|'})


In [ ]:
y = final_data['abnormal_cogs']
model_5 = sm.OLS(y,X).fit()
#print(model_5.summary())

In [ ]:
reg_cogs = pd.concat((model_5.params, model_5.tvalues,model_5.pvalues), axis=1)

reg_cogs = reg_cogs.rename(columns={0: 'Coef.', 
                                  1: 't',
                                  2:'p>|t|'})

In [ ]:
y = abs(final_data['abnormal_ocf'])+abs(final_data['abnormal_cogs'])+abs(final_data['abnormal_prod'])+abs(final_data['abnormal_inv'])+abs(final_data['abnormal_disexp'])
model_6 = sm.OLS(y,X).fit()
#print(model_6.summary())

In [ ]:
reg_rm = pd.concat((model_6.params, model_6.tvalues,model_6.pvalues), axis=1)

reg_rm = reg_rm.rename(columns={0: 'Coef.', 
                                  1: 't',
                                  2:'p>|t|'})

In [ ]:
final_am = pd.merge(final_1,
                         right = ams_all,
                         on = ['tic','year'])

final_am['abnormal_ocf']=final_am['abnormal_ocf']*100
final_am['abnormal_cogs']=final_am['abnormal_cogs']*100
final_am['abnormal_disexp']=final_am['abnormal_disexp']*100
final_am['abnormal_inv']=final_am['abnormal_inv']*100
final_am['abnormal_prod']=final_am['abnormal_prod']*100
final_am['am']=final_am['am']*100
final_am['gov']=np.log(final_am['governance_score']+1)

In [ ]:
eda = pd.DataFrame(final_am.describe())

In [ ]:
final_am = pd.get_dummies(final_am,columns=['year'])
final_am

In [ ]:
y = abs(final_am['am'])
model_7 = sm.OLS(y,X).fit()
#print(model_7.summary())

In [ ]:
reg_am = pd.concat((model_7.params, model_7.tvalues,model_7.pvalues), axis=1)

reg_am = reg_am.rename(columns={0: 'Coef.', 
                                  1: 't',
                                  2:'p>|t|'})

In [ ]:
X = final_am[['coverage','ptb','roa','size','cycle',
                'year_2011','year_2012','year_2013',
                'year_2014','year_2015','year_2016',
                'year_2017']]

X = sm.add_constant(X)
rm = abs(final_am['abnormal_ocf'])+abs(final_am['abnormal_cogs'])+abs(final_am['abnormal_prod'])+abs(final_am['abnormal_inv'])+abs(final_am['abnormal_disexp'])

am = abs(final_am['am'])

y = rm/(rm+am)

model_8 = sm.OLS(y,X).fit()
#print(model_8.summary())

In [ ]:
reg_rms = pd.concat((model_8.params, model_8.tvalues,model_8.pvalues), axis=1)

reg_rms = reg_rms.rename(columns={0: 'Coef.', 
                                  1: 't',
                                  2:'p>|t|'})

In [ ]:
# Create a Pandas Excel writer using XlsxWriter as the engine.

writer = pd.ExcelWriter('analyst_coverage_reg_results.xlsx')

# Write each dataframe to a different worksheet.
eda.to_excel(writer,sheet_name = 'EDA')
reg_ocf.to_excel(writer, sheet_name='ocf')
reg_disexp.to_excel(writer, sheet_name='disexp')
reg_inv.to_excel(writer, sheet_name='inv')
reg_prod.to_excel(writer, sheet_name='prod')
reg_cogs.to_excel(writer, sheet_name='cogs')
reg_rm.to_excel(writer, sheet_name='rm')
reg_am.to_excel(writer, sheet_name='am')
reg_rms.to_excel(writer, sheet_name='rms')

# Close the Pandas Excel writer and output the Excel file.
writer.save()

files.download('analyst_coverage_reg_results.xlsx')

# REGRESSION WITH GOVERNANCE

In [ ]:
X = final_am[['coverage','gov','ptb',
              'roa','size','cycle',
              'year_2011','year_2012','year_2013',
              'year_2014','year_2015','year_2016',
              'year_2017']]

X = sm.add_constant(X)

rm = abs(final_am['abnormal_ocf'])+abs(final_am['abnormal_cogs'])+abs(final_am['abnormal_prod'])+abs(final_am['abnormal_inv'])+abs(final_am['abnormal_disexp'])

am = abs(final_am['am'])

rms = rm/(rm+am)

model_rm= sm.OLS(rm,X).fit()
model_am= sm.OLS(am,X).fit()
model_rms= sm.OLS(rms,X).fit()

X_gov = final_am[['gov','ptb',
              'roa','size','cycle',
              'year_2011','year_2012','year_2013',
              'year_2014','year_2015','year_2016',
              'year_2017']]

X_gov = sm.add_constant(X_gov)


g_rm= sm.OLS(rm,X_gov).fit()
g_am= sm.OLS(am,X_gov).fit()
g_rms= sm.OLS(rms,X_gov).fit()

In [ ]:
re_rm = pd.concat((model_rm.params, model_rm.tvalues,model_rm.pvalues), axis=1)
re_am = pd.concat((model_am.params, model_am.tvalues,model_am.pvalues), axis=1)
re_rms = pd.concat((model_rms.params, model_rms.tvalues,model_rms.pvalues), axis=1)

re_ocf = re_ocf.rename(columns={0: 'Coef.', 
                                  1: 't',
                                  2:'p>|t|'})

re_cogs = re_cogs.rename(columns={0: 'Coef.', 
                                  1: 't',
                                  2:'p>|t|'})

re_prod = re_prod.rename(columns={0: 'Coef.', 
                                  1: 't',
                                  2:'p>|t|'})

re_dis = re_dis.rename(columns={0: 'Coef.', 
                                  1: 't',
                                  2:'p>|t|'})

re_inv = re_inv.rename(columns={0: 'Coef.', 
                                  1: 't',
                                  2:'p>|t|'})

re_rm = re_rm.rename(columns={0: 'Coef.', 
                                  1: 't',
                                  2:'p>|t|'})

re_am = re_am.rename(columns={0: 'Coef.', 
                                  1: 't',
                                  2:'p>|t|'})



re_rms = re_rms.rename(columns={0: 'Coef.', 
                                  1: 't',
                                  2:'p>|t|'})


In [ ]:
g_rm = pd.concat((g_rm.params, g_rm.tvalues,g_rm.pvalues), axis=1)
g_am = pd.concat((g_am.params, g_am.tvalues,g_am.pvalues), axis=1)
g_rms = pd.concat((g_rms.params, g_rms.tvalues,g_rms.pvalues), axis=1)

g_ocf = g_ocf.rename(columns={0: 'Coef.', 
                                  1: 't',
                                  2:'p>|t|'})

g_cogs = g_cogs.rename(columns={0: 'Coef.', 
                                  1: 't',
                                  2:'p>|t|'})

g_prod = g_prod.rename(columns={0: 'Coef.', 
                                  1: 't',
                                  2:'p>|t|'})

g_dis = g_dis.rename(columns={0: 'Coef.', 
                                  1: 't',
                                  2:'p>|t|'})

g_inv = g_inv.rename(columns={0: 'Coef.', 
                                  1: 't',
                                  2:'p>|t|'})

g_rm = g_rm.rename(columns={0: 'Coef.', 
                                  1: 't',
                                  2:'p>|t|'})

g_am = g_am.rename(columns={0: 'Coef.', 
                                  1: 't',
                                  2:'p>|t|'})



g_rms = g_rms.rename(columns={0: 'Coef.', 
                                  1: 't',
                                  2:'p>|t|'})


In [ ]:
# Create a Pandas Excel writer using XlsxWriter as the engine.

writer = pd.ExcelWriter('reg_results_gov_ac.xlsx')

# Write each dataframe to a different worksheet.
re_rm.to_excel(writer, sheet_name='rm')
re_am.to_excel(writer, sheet_name='am')
re_rms.to_excel(writer, sheet_name='rms')

# Close the Pandas Excel writer and output the Excel file.
writer.save()

files.download('reg_results_gov_ac.xlsx')

In [ ]:
# Create a Pandas Excel writer using XlsxWriter as the engine.

writer = pd.ExcelWriter('reg_results_gov.xlsx')

# Write each dataframe to a different worksheet.
g_rm.to_excel(writer, sheet_name='rm')
g_am.to_excel(writer, sheet_name='am')
g_rms.to_excel(writer, sheet_name='rms')

# Close the Pandas Excel writer and output the Excel file.
writer.save()

files.download('reg_results_gov.xlsx')

In [ ]:
final_am['gov_dum'] = np.where(final_am['gov']>=4.181868, 1, 0)
final_am['gov_cov'] = final_am['gov']*final_am['coverage']
final_am.describe()

In [ ]:
X_dum = final_am[['coverage','gov_dum','gov_cov','ptb',
              'roa','size','cycle',
              'year_2011','year_2012','year_2013',
              'year_2014','year_2015','year_2016',
              'year_2017']]

X_dum = sm.add_constant(X_dum)


rm = abs(final_am['abnormal_ocf'])+abs(final_am['abnormal_cogs'])+abs(final_am['abnormal_prod'])+abs(final_am['abnormal_inv'])+abs(final_am['abnormal_disexp'])

am = abs(final_am['am'])

rms = rm/(rm+am)

model_rm_d= sm.OLS(rm,X_dum).fit()
model_am_d= sm.OLS(am,X_dum).fit()
model_rms_d= sm.OLS(rms,X_dum).fit()


In [ ]:
m_rm = pd.concat((model_rm_d.params, model_rm_d.tvalues,model_rm_d.pvalues), axis=1)
m_am = pd.concat((model_am_d.params, model_am_d.tvalues,model_am_d.pvalues), axis=1)
m_rms = pd.concat((model_rms_d.params, model_rms_d.tvalues,model_rms_d.pvalues), axis=1)

m_rm = m_rm.rename(columns={0: 'Coef.', 
                                  1: 't',
                                  2:'p>|t|'})

m_am = m_am.rename(columns={0: 'Coef.', 
                                  1: 't',
                                  2:'p>|t|'})



m_rms = m_rms.rename(columns={0: 'Coef.', 
                                  1: 't',
                                  2:'p>|t|'})


In [ ]:
# Create a Pandas Excel writer using XlsxWriter as the engine.

writer = pd.ExcelWriter('reg_results_gov_dum.xlsx')

# Write each dataframe to a different worksheet.

m_rm.to_excel(writer, sheet_name='rm')
m_am.to_excel(writer, sheet_name='am')
m_rms.to_excel(writer, sheet_name='rms')

# Close the Pandas Excel writer and output the Excel file.
writer.save()

files.download('reg_results_gov_dum.xlsx')

In [ ]:
final_am['rm'] = abs(final_am['abnormal_ocf'])+abs(final_am['abnormal_cogs'])+abs(final_am['abnormal_prod'])+abs(final_am['abnormal_inv'])+abs(final_am['abnormal_disexp'])

final_am['am'] = abs(final_am['am'])

final_am['rms'] = rm/(rm+am)

final_am.describe()


In [ ]:
from scipy.stats import pearsonr
import numpy as np
regressors = final_am[['am',
                       'abnormal_ocf',	
                       'abnormal_cogs',
                       'abnormal_disexp',
                       'abnormal_inv',
                       'abnormal_prod',
                       'rm',
                       'rms',                       
                       'coverage',
                       'gov',
                       'roa',
                       'cycle',
                       'ptb',
                       'size']]
                       
rho = regressors.corr()
pval = regressors.corr(method=lambda x, y: pearsonr(x, y)[1]) - np.eye(*rho.shape)
p = pval.applymap(lambda x: ''.join(['*' for t in [0.01,0.05,0.1] if x<=t]))
corr = rho.round(2).astype(str) + p

corr

In [ ]:
corr.to_csv('corr.csv') 
files.download('corr.csv')

In [ ]:
final_am.to_csv('final_data.csv')
files.download('final_data.csv')